In [ ]:
import streamlit as st
from datetime import datetime, date
import pandas as pd

def load_appointments(file_path):
    return pd.read_excel(file_path)

def preprocess(df):
    df = df[df['Status'] == 'Complete'].copy()
    df['Start'] = pd.to_datetime(df['Start'])
    df['End'] = pd.to_datetime(df['End'])
    df['Created At'] = pd.to_datetime(df['Created At']).dt.date
    df['Original_Date'] = df['Start'].dt.date
    df['Duration'] = (df['End'] - df['Start']).dt.total_seconds() / 60
    df = df.reset_index(drop=True)
    df['Appt_ID'] = df.index
    return df[['Appt_ID', 'Medication', 'Duration', 'Original_Date', 'Created At']].dropna()

def get_clinic_days_same_month(date):
    first_day = date.replace(day=1)
    next_month = (first_day + pd.DateOffset(days=32)).replace(day=1)
    last_day = next_month - pd.DateOffset(days=1)
    clinic_days = []
    current = first_day
    while current <= last_day:
        if current.weekday() in [1, 2, 3]:
            clinic_days.append(current.date())
        current += pd.DateOffset(days=1)
    return sorted(clinic_days)

def allocate_appointments(df):
    CLINIC_MINUTES = 540
    allocations = []
    unassigned = []
    for medication, group in df.groupby('Medication'):
        appts = group.sort_values(by=['Original_Date', 'Duration'], ascending=[True, False]).to_dict('records')
        schedule = {}
        for appt in appts:
            original_date = appt['Original_Date']
            duration = appt['Duration']
            same_month_days = get_clinic_days_same_month(original_date)
            same_month_days.sort(key=lambda d: abs((d - original_date).days))
            assigned = False
            for day in same_month_days:
                if day not in schedule:
                    schedule[day] = {'total': 0, 'items': []}
                if schedule[day]['total'] + duration <= CLINIC_MINUTES:
                    appt['Assigned_Date'] = day
                    schedule[day]['items'].append(appt)
                    schedule[day]['total'] += duration
                    assigned = True
                    break
            if not assigned:
                unassigned.append(appt)
        for day, data in schedule.items():
            allocations.extend(data['items'])
    return pd.DataFrame(allocations), pd.DataFrame(unassigned)

def rebalance_schedule_strict_global(assigned_df):
    CLINIC_MINUTES = 540
    assigned_df = assigned_df.sort_values(by=['Medication', 'Assigned_Date', 'Duration'], ascending=[True, True, False])
    rebalanced_rows = []
    day_totals = {}
    for _, row in assigned_df.iterrows():
        day = row['Assigned_Date']
        day_totals[day] = day_totals.get(day, 0) + row['Duration']
    for _, row in assigned_df.iterrows():
        row_dict = row.to_dict()
        day = row_dict['Assigned_Date']
        row_dict['Days_Moved'] = (day - row_dict['Original_Date']).days
        rebalanced_rows.append(row_dict)
    return pd.DataFrame(rebalanced_rows), day_totals

def suggest_appointment_slots(day_totals, proposed_date, duration, created_at=None):
    CLINIC_MINUTES = 540
    created_at = created_at or date.today()
    month_start = proposed_date.replace(day=1)
    if proposed_date.month == 12:
        next_month = proposed_date.replace(year=proposed_date.year+1, month=1, day=1)
    else:
        next_month = proposed_date.replace(month=proposed_date.month+1, day=1)
    month_end = next_month - timedelta(days=1)
    clinic_days = []
    current = month_start
    while current <= month_end:
        if current.weekday() in [1, 2, 3] and current >= created_at:
            clinic_days.append(current)
        current += timedelta(days=1)
    available_days = []
    for day in clinic_days:
        total_used = day_totals.get(day, 0)
        remaining = CLINIC_MINUTES - total_used
        if remaining >= duration:
            score = abs((day - proposed_date).days)
            available_days.append((day, remaining, score))
    available_days.sort(key=lambda x: (x[2], -x[1]))
    return available_days[:3]

st.set_page_config(page_title="Appointment Scheduler", layout="centered")
st.title("💊 Clinic Appointment Scheduler")

@st.cache_data
def load_and_rebalance():
    df = load_appointments("Appointments_Vegas.xlsx")
    df_clean = preprocess(df)
    assigned_df, _ = allocate_appointments(df_clean)
    if 'Assigned_Date' not in assigned_df.columns:
        assigned_df['Assigned_Date'] = assigned_df['Original_Date']
    rebalanced_df, day_totals = rebalance_schedule_strict_global(assigned_df)
    return day_totals

day_totals = load_and_rebalance()

with st.form("appointment_form"):
    st.subheader("Proposed Appointment")
    proposed_date = st.date_input("Desired Date", value=date.today())
    duration = st.number_input("Duration (minutes)", min_value=1, max_value=540, value=45)
    use_created = st.checkbox("Specify Created At Date?")
    if use_created:
        created_at = st.date_input("Created At Date", value=date.today())
    else:
        created_at = None
    submitted = st.form_submit_button("Find Best Slots")

if submitted:
    st.markdown("### 📅 Suggested Appointment Dates")
    suggestions = suggest_appointment_slots(
        day_totals=day_totals,
        proposed_date=proposed_date,
        duration=duration,
        created_at=created_at
    )
    if suggestions:
        for d, remaining, _ in suggestions:
            st.success(f"{d.strftime('%A, %Y-%m-%d')} — {remaining} minutes remaining")
    else:
        st.error("No available slots under current constraints.")
